# Miembros de Grupos

* PRODUCTOS
  * autor registrante
     * Institución
     * Grupo


## Modules, functions and data

In [1]:
import wosplus as wp

In [320]:
import pandas as pd
import numpy as np
import time
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [711]:
#https://stackoverflow.com/a/57915246/2268280
#https://stackoverflow.com/a/50404592/2268280
#https://stackoverflow.com/a/16991985/2268280
import json
import datetime
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, datetime.datetime):
            return str(obj)
        return super(NpEncoder, self).default(obj)

In [770]:
def iloc_0(df):
    if not df.empty:
        df=df.iloc[0]
    else:
        df=pd.Series(dtype=str)
    return df
def get_author(RH,other=False,
               document_type={'C':"cédula", 'P':"passport", 'E':"cédula de extranería", 'T':"tarjeta de identidad"},
              roles={"ORI":"Advisor"}):
    #DEBUG:
    if RH.get("TPO_PARTICIPACION") and RH.get("TPO_PARTICIPACION")!="ORI":
        roles[RH.get("TPO_PARTICIPACION"):RH.get("TPO_PARTICIPACION")]
    if RH.get("TXT_TOTAL_NAMES"):
        aud={
            "full_name":str(RH.get("TXT_TOTAL_NAMES")),
            "first_names": str(RH.get("TXT_NAMES_RH")),
            "first_last_name": str(RH.get("TXT_PRIM_APELL")), #DEBUG: last_names
            "second_last_name": str(RH.get("TXT_SEG_APELL")), 
            "intials":''.join([s[0] for s in str(RH.get("TXT_NAMES_RH")).split()]),
            "aliases":[str(RH.get("TXT_CITACION_BIBLIO"))],
        }
    else:
        aud={}
    if RH.get("TPO_SEXO"):
        aud["sex"]=str(RH.get("TPO_SEXO"))
    if RH.get("TPO_NACIONALIDAD"):  
        aud["nacionality"]=str(RH.get("TPO_NACIONALIDAD")) #privado
    if RH.get("TPO_ESTADO_CIVIL"):
        aud["marital_status"]=str(RH.get("TPO_ESTADO_CIVIL")) #privado
    if RH.get("DTA_NACIM"):
        aud["date_of_birth"]=str(RH.get("DTA_NACIM")) #privado
    
    if RH.get("SGL_PAIS_NACIM"):
        PS=ps[ps['SGL_PAIS']==RH.get("SGL_PAIS_NACIM")]
        PS=iloc_0(PS)
        aud["country_of_birth"]=str(PS.get("TXT_NME_PAIS")) #privado
    if RH.get("COD_MUN_NACIM"):
        MP=mp[ (mp["COD_RH_MUNICIPIO"]==RH.get("COD_RH_MUN_NACIM")) & ( mp["COD_MUNICIPIO"]==RH.get("COD_MUN_NACIM"))]
        MP=iloc_0(MP)
        aud["town_of_birth"]=str(MP.get("TXT_NME_MUNICIPIO"))
    if RH.get("COD_RH"):
        aud["external_ids"]=[
                         {"value":str(RH.get("COD_RH")),
                          "source":"cvlac"
                         }]
    
    if RH.get("COD_ORCID") and not pd.isna(RH.get("COD_ORCID")):
        aud["external_ids"].append({"value":RH.get("COD_ORCID"),
                                    "source":"orcid"
                                   })
    if RH.get("AUTOR_ID_SCP") and not pd.isna(RH.get("AUTOR_ID_SCP")):
        aud["external_ids"].append({"value":RH.get("AUTOR_ID_SCP"),
                                    "source":"orcid"
                                   })
    if RH.get("NRO_DOCUMENTO_IDENT") and not pd.isna(RH.get("NRO_DOCUMENTO_IDENT")):
        aud["document"]=[{"value":RH.get("NRO_DOCUMENTO_IDENT"),
                      "type":document_type.get( RH.get("TPO_DOCUMENTO_IDENT") )}]

        if not pd.isna(RH.get("NRO_PASAPORTE")) and "passport" in [d.get("type") for d in aud["document"] ] :
            aud["document"].append({"value":RH.get("NRO_PASAPORTE"),
                                "type":"passport"
                               })
    if not other:
        aud["registrant"]=[{"value":True,
                              "source":"scienti"}] #DEBUG Unificar autor de correspondencia, envio preprint
        aud["role"]="author"
    else:
        aud["registrant"]=[{"value":False,
                              "source":"scienti"}] #DEBUG Unificar autor de correspondencia, envio preprint
        aud["role"]=roles.get(str(RH.get("TPO_PARTICIPACION")))
    return aud        

In [3]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

from IPython.display import JSON
class JSON_notebook(object): #JSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 100px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)
        
from IPython.display import Markdown
def jsonviewer(d):
    f=open('file.json','w')
    json.dump(d,f)
    f.close()
    print('open in firefox new tab:')
    return Markdown('[file.json](./file.json)')        

In [554]:
dtype={
'COD_RH':str,
'NRO_ID_GRUPO':str,
'COD_RH_MUNICIPIO':str,
'COD_INST':str,
'COD_INST_AVALA':str,
'COD_INST_MACRO':str,
'COD_INSTITUCION':str,         
'COD_INST_OTRO':str,
'ID_DEPARTAMENTO':str,
'COD_DEPA_DANE': str,
'COD_MUNI_DANE':str,
'COD_RH_OTRO':str,
'COD_RH_COAUTOR':str
 }
#converters={'COD_INST_AVALA':lambda s:'0' if pd.isna(s) else str(s)},

In [562]:
sc=wp.wosplus('drive.cfg')

## PRODUCTOS

In [162]:
cv=sc.read_drive_excel('EN_PRODUCTO.xlsx',dtype=dtype)
rh=sc.read_drive_excel('EN_RECURSO_HUMANO.xlsx',dtype=dtype)

In [37]:
tp=sc.read_drive_excel('EN_TIPO_PRODUCTO.xlsx',dtype=dtype)

In [136]:
gr=sc.read_drive_excel('GRUPLAC_EN_PRODUCTO_GR.xlsx',dtype=dtype)

In [ ]:
PPC=sc.read_drive_excel('RE_PRODUCTO_PALABRA_CLA.xlsx',dtype=dtype)
PC=sc.read_drive_excel('EN_PALABRA_CLAVE.xlsx',dtype=dtype)

In [301]:
prh=sc.read_drive_excel('RE_PRODUCTO_RECURSO_HUM_OTRO.xlsx',dtype=dtype)

In [ ]:
ps=sc.read_drive_excel('EN_PAIS.xlsx',dtype=dtype)
dp=sc.read_drive_excel('EN_DEPARTAMENTO.xlsx',dtype=dtype)
mp=sc.read_drive_excel('EN_MUNICIPIO.xlsx',dtype=dtype)

In [406]:
psa=sc.read_drive_excel('RE_PRODUCTO_SECTOR_APL.xlsx',dtype=dtype)

In [407]:
sa=sc.read_drive_excel('EN_SECTOR_APLICACION.xlsx',dtype=dtype)

In [466]:
gi=sc.read_drive_excel('GRUPLAC_RE_GRUPO_INSTITUCION.xlsx',dtype=dtype)

In [491]:
it=sc.read_drive_excel('EN_INSTITUCION.xlsx',dtype=dtype)

In [522]:
GR=sc.read_drive_excel('gruplac_grupo_pesquisa.xlsx',dtype=dtype)

In [538]:
ac=sc.read_drive_excel('EN_AREA_CONOCIMIENTO.xlsx',dtype=dtype)

In [595]:
prg=sc.read_drive_excel('GRUPLAC_EN_PROGRAMA_COLCIENCIAS.xlsx',dtype=dtype)

In [653]:
NO_CORRER=True
if not NO_CORRER:
    df=pd.read_pickle('/data/scienti/cvlac.pkl.gz')
    print(df.shape)

In [585]:
if not NO_CORRER:
    primary_tables=[]
    sgl_cat=list(df['SGL_CATEGORIA'].dropna().unique())
    new_cat=set( sgl_cat )
    for t in df['table'].unique():
        t_cat=df[df['table']==t].get('SGL_CATEGORIA')
        if t_cat is not None:
            nc=new_cat.difference( t_cat.dropna().unique() )
            if len(new_cat)>len(nc):
                new_cat=nc
                primary_tables.append(t)

                print(f'primary → {t}')
        #print(f'{t}')
else:
    primary_tables=['EN_PRODUCTO',
 'EN_EVENTO',
 'EN_SECRETO_INDUSTRIAL',
 'EN_PATENTE',
 'EN_PROD_ARTISTICA_DETALLE',
 'EN_PROYECTO',
 'EN_REGISTRO',
 'EN_RED']
    sgl_cat=['TP-TP_B', 'ART-ART_A1', 'ART-ART_A2', 'ART-ART_D', 'ART-ART_B',
       'ART-GC_ART', 'IPP-00', 'GC_M-00', 'ART-ART_C', 'TM-TM_A',
       'ART-00', 'TP-00', 'TM-TM_B', 'INF-00', 'TD-TD_A', 'TM-00',
       'TD-TD_B', 'TD-00', 'TP-TP_A', 'IPP-IPP', 'IFI-IFI', 'WP-WP',
       'ERL-ERL', 'GC_I-00', 'CON_CT-00', 'SF-00', 'PP-00', 'GC_I-GC',
       'IFI-00', 'CON_CT-CON_CT', 'PI-PI_A', 'PI-00', 'DI-00', 'RN-00',
       'GC_M-EX', 'GC_M-GC', 'ERL-00', 'SF-EX', 'GC_I-EX', 'GC_V-GC',
       'BOL-00', 'GC_V-00', 'RNPC-RNPC', 'BOL-BOL', 'INF-INF', 'IG-IG_A1',
       'EBT-00', 'SF-SF_A', 'NSG-NSG', 'DI-DI_A', 'IG-00', 'RN-RNL_B',
       'WP-00', 'CON_AAD-CON_AAD', 'IG-IG_A2', 'GC_V-EX', 'IG-IG_B2',
       'EBT-EBT_A', 'PP-PP_A', 'ECI-00', 'ICC-ICC_A', 'SD-EX', 'SD-SD',
       'IG-IG_B1', 'RNPC-00', 'DI-DI_B', 'VA-00', 'RN-RNT', 'CON_AAD-00',
       'ICC-00', 'RN-RNR', 'EBT-EBT_B', 'VV-VV_B2', 'RNPL-00', 'VV-VV_A2',
       'VV-VV_B3', 'VV-VV_B4', 'GC_A-GC', 'N-N_A2', 'CT-CT', 'N-N_B',
       'NRC-NRC_B', 'NRC-NRC_A', 'N-N_C', 'PAU-PAU', 'N-N_D', 'N-GC_N',
       'N-00', 'PN-PN', 'AL-AL', 'NRC-00', 'AL-00', 'PAU-00', 'N-N_A1',
       'CT-00', 'EC-EC_B', 'EC-00', 'EC-EX', 'EC-EC_A', 'ECA-ECA_A',
       'APO-00', 'ECA-ECA_B', 'EPC-00', 'EPC-EPC', 'EPC-EX', 'APO-APO',
       'TC-00', 'ECA-00', 'TC-EX', 'TC-TC_C', 'TC-TC_B', 'TC-TC_A',
       'SE-00', 'SE-SE', 'PIV-00', 'PMU-00', 'PIV-EX', 'PIV-PA4',
       'PIV-PB4', 'PIV-PA3', 'PMU-MA4', 'PIV-PA2', 'PMU-EX', 'PIV-PB2',
       'PIV-PB5', 'PIV-PA1', 'PIV-PC', 'AAD-00', 'AAD-AAD_C', 'AAD-AAD_B',
       'AAD-AAD_A', 'AAD-AAD_A1', 'PID-00', 'PF-00', 'PE-PE', 'PE-00',
       'PCC-00', 'PID-PID_C', 'PID-PID_B', 'EPA-EPA', 'PPC-PPC', 'PPC-EX',
       'PID-EX', 'PCC-PCC', 'PID-PID_A', 'PF-EX', 'PCC-EX', 'PF-PF_A',
       'PPC-00', 'PIC-PIC_A', 'PIC-00', 'PIC-PIC_C', 'EPA-EX', 'PF-PF_B',
       'EPA-00', 'PIC-PIC_B', 'REG_G-00', 'REG_SF-EX', 'REG_SF-00',
       'MR-MR', 'REG_G-CG', 'REG_G-EX', 'RC-00', 'RC-RC_A']

In [21]:
gr[gr['SGL_CATEGORIA']=='ART-ART_A1'].shape[0]

5536

In [22]:
for i in range(gr[gr['SGL_CATEGORIA']=='ART-ART_A1'].shape[0]):
    art_gr=gr[gr['SGL_CATEGORIA']=='ART-ART_A1'].iloc[i]

    art_cv=cv[ ( (cv['COD_RH']==art_gr['COD_RH']) & (cv['COD_PRODUCTO']==art_gr['COD_PRODUCTO']) )].iloc[0]
    if isinstance(art_cv['TXT_DOI'],str):
        print(i)
        break

16


In [35]:
jsonviewer( [art_cv.dropna().astype(str).to_dict()] )

open in firefox new tab:


[file.json](./file.json)

In [646]:
JSON(art_cv.dropna().astype(str).to_dict())

NameError: name 'art_cv' is not defined

In [ ]:
OPTIMIZE_STORAGE=True
if OPTIMIZE_STORAGE:
    borrar='_BORRAR'
else:
    borrar=''
prs=[]
i=888#117294#0#340454#666#

prmax=10
#if True:
for i in cv.index:
    #if i==101:#20:
    #    break
    pr={}
    if i%100==0:
        print(str(i).zfill(6),end='\r')    
    cvi=cv.loc[i]
    DEBUG=False
    #Tipo de Producto
    TP=tp[tp['COD_TIPO_PRODUCTO']==cvi.get('COD_TIPO_PRODUCTO')]
    TP=iloc_0(TP)

    #author
    RH=rh[rh["COD_RH"]==cvi.get("COD_RH")]
    RH=iloc_0(RH)

    #coauthor/advisor
    PRH=prh[(prh['COD_RH']==cvi['COD_RH']) & (prh['COD_PRODUCTO']==cvi['COD_PRODUCTO'])]
    PRH=iloc_0(PRH)

    #Sector Aplicación
    PSA=psa[ (psa['COD_RH']==cvi['COD_RH']) & (psa['COD_PRODUCTO']==cvi['COD_PRODUCTO']) ]
    PSA=iloc_0(PSA)

    SA=sa[sa['COD_SECTOR_APLICACION']==PSA.get('COD_SECTOR_APLICACION')]
    SA=iloc_0(SA)

    #Product in Group
    GRR=gr[ (gr['COD_RH']==cvi['COD_RH']) & (gr['COD_PRODUCTO']==cvi['COD_PRODUCTO']) ]
    GRR=iloc_0(GRR)
    pr["year_published"]=GRR.get('NRO_ANO_PRESENTA')
    pr["month_published"]=GRR.get('NRO_MES_PRESENTA')

    #Group code number o
    GI=gi[gi['NRO_ID_GRUPO']==GRR.get('NRO_ID_GRUPO')]
    GI=iloc_0(GI)

    #Institute 
    IT=it[it['COD_INST']==GI.get('COD_INST')]
    IT=iloc_0(IT)

    #Group information
    GP=GR[GR['NRO_ID_GRUPO']==GRR.get('NRO_ID_GRUPO')]
    GP=iloc_0(GP)

    DP=dp[dp['ID_DEPARTAMENTO']==IT.get('SGL_DEPARTAMENTO')]
    DP=iloc_0(DP)
    PS=ps[ps['SGL_PAIS']==DP.get('SGL_PAIS')]
    PS=iloc_0(PS)


    GRP=gr[(gr['COD_RH']==cvi.get('COD_RH')) & (gr['COD_PRODUCTO']==cvi.get('COD_PRODUCTO'))]
    GRP=iloc_0(GRP)

    GRU=GR[GR["NRO_ID_GRUPO"]==GRP.get("NRO_ID_GRUPO")]
    GRU=iloc_0(GRU)

    PRG=prg[prg['COD_PROGRAMA']==GRU.get('COD_PROGRAMA')]
    PRG=iloc_0(PRG)

    AC=ac[ac["COD_AREA_CONOCIMIENTO"]==GRU.get("COD_AREA_CONHEC")]
    AC=iloc_0(AC)

    if DEBUG: print('general metadata')

    pr['source_checked']=[]
    pr['source_checked'].append(
        {'source': 'Scienti',
         'COD_RH': cvi.get('COD_RH'),
         'COD_PRODUCTO':cvi.get('COD_PRODUCTO')
        })

    pr['title']=cvi.get('TXT_NME_PROD')

    if not pd.isna( cvi.get('TXT_RESUMEN_PROD') ):
        pr['abstract']=cvi.get(f'TXT_RESUMEN_PROD{borrar}')
    else:
        pr['abstract']=''

    if not pd.isna( cvi.get('TXT_OBSERV_PROD') ):
        pr['notes']=cvi.get(f'TXT_OBSERV_PROD{borrar}')
    else:
        pr['notes']=''

    pr["lang"]=str(cvi.get("SGL_IDIOMA"))

    pr['product_subtype']=[{'title':TP.get('TXT_NME_TIPO_PRODUCTO')}]
    if not pd.isna(cvi.get('SGL_CATEGORIA')):
        pr['product_subtype'][0]['SGL_CATEGORIA']=cvi.get('SGL_CATEGORIA')
    else:
        pr['product_subtype'][0]['SGL_CATEGORIA']=''  
    #TODO → ADD GRUPLAC PRODUCT information
    #TODO → Add specific 'product_subtype' information

    if DEBUG: print('topics')

    tp_names=[]
    stop=0
    while not TP.empty:
        TP=tp[tp['COD_TIPO_PRODUCTO']==TP.get('COD_TIPO_PRODUCTO_PADRE')]
        if not TP.empty:
            TP=TP.iloc[0]
        else:
            TP=pd.Series(dtype=str)
        if not pd.isna(TP.get('TXT_NME_TIPO_PRODUCTO')):
            tp_names.append(TP.get('TXT_NME_TIPO_PRODUCTO'))
            tp_names.reverse()
        stop+=1
        if stop>10:
            break

    tp_list=[]
    for i in range(len(tp_names)):
        tp_list.append( {'title':tp_names[i],
                         'level':i})

    pr["product_type"]=tp_list

    pc=PPC[(PPC['COD_RH']==cvi.get('COD_RH')) & (PPC['COD_PRODUCTO']==cvi.get('COD_PRODUCTO'))]

    lpc=[]
    for i in pc['COD_PALABRA_CLAVE'].values: 
        lpc.append( PC[(PC['COD_RH']==cvi.get('COD_RH')) & (PC['COD_PALABRA_CLAVE']==i)].iloc[0].to_dict())

    pr["keywords"]=[d.get('TXT_NME_PALABRA_CLAVE') for d in lpc]

    if DEBUG: print('general authors metadata')

    pr["authors"]=[]


    pr["authors"].append(get_author(RH))


    if not pd.isna(PRH.get("COD_RH_COAUTOR")) and len(PRH.get("COD_RH_COAUTOR"))==10:
        RH=rh[rh["COD_RH"]==PRH.get("COD_RH_COAUTOR")]
        RH=iloc_0(RH)
        if RH.empty:
            RH["COD_RH"]=PRH["COD_RH"]
        #    RH["TXT_NAMES_RH"]=''
        RH["TPO_PARTICIPACION"]=PRH["TPO_PARTICIPACION"]
        pr["authors"].append(get_author(RH,other=True))

    pr["profile_description"]=RH.get(f"TXT_DESC_PERFIL{borrar}")


    if DEBUG: print('sector de aplicación')
    isa=True
    if SA.empty:
        isa=False
    sal=[]
    stop=0
    while isa:
        stop+=1
        if SA.get('TXT_NME_SECTOR_APLIC'):
            sal.append(SA.get('TXT_NME_SECTOR_APLIC'))
        else:
            isa=False
        if SA.get('COD_SECTOR_APLICACION') and not SA.get('COD_SECTOR_APLICACION')!=SA.get('COD_SECT_APLIC_PADRE'):
            isa=False
        else:
            SA=sa[sa['COD_SECTOR_APLICACION']==SA.get('COD_SECT_APLIC_PADRE')]
            SA=iloc_0(SA)
        if stop>10:
            print("to many PADREs in COD_SECT_APLIC_PADRE")
            break
    sal.reverse()        

    sall=[]
    for i in range(len(sal)):
        sall.append({"title":sal[i],"level":i})

    pr["application_sector"]=sall


    if DEBUG: print('affiliation university')

    if IT.empty:
        aff={}
    else:
        aff={
        "name":str(IT.get('NME_INST')),
        'acronym':str(IT.get('SGL_INST')),
        'addresses': [{'line_1': str(IT.get("TXT_DIRECCION")),
                       "telephone":str(IT.get("TXT_TELEFONO")),
                       "email":str(IT.get("TXT_EMAIL"))
                                       }],
        'city': str(IT.get("TXT_CIUDAD_INST")), #town
        'date_constitution': str(IT.get("DTA_CONSTITUCION")),                  
        'external_urls': [{'source': 'scienti',
                                     'url': str(IT.get("URL_HOME_PAGE"))}],
                         # EN_INSTITUCION_OTRA → EN_INSTITUCION
        'external_ids': [{'source': 'NIT', 'value': str(IT.get("TXT_NIT"))}],
        'state': str(DP.get("TXT_NME_DEPARTAMENTO")),
        'state_code': {"value":str(DP.get("COD_DEPA_DANE")),
                       "type":"DANE"}, #https://www.fopep.gov.co/wp-content/uploads/2019/02/Tabla-C%C3%B3digos-Dane.pdf
        'country': PS.get(str("TXT_NME_PAIS"))   
        }

    if DEBUG: print('affiliation group')

    if GRU.empty:
        branch={}
    else:
        branch={
        "name":str(GRU.get("NME_GRUPO")),                                      
        "type":"group",
        'external_urls': [{ # → NRO_ID_GRUPO
        'source': 'scienti',
        'url': f'https://scienti.colciencias.gov.co/gruplac/jsp/visualiza/visualizagr.jsp?nro={GRU.get("NRO_ID_GRUPO")}'
                        }],
        'external_ids': [{'source': 'gruplac', 'id': str(GRU.get("COD_ID_GRUPO"))}],                                
        "formation_year":GRU.get("ANO_FORMACAO"),
        "month_formation_year":GRU.get("MES_FORMACAO"),
        #'subjects': [{'source': 'area_ocde','subjects': ['Ciencias Sociales']},
        #                                     {'source': 'subarea_ocde', 'subjects': ['Ciencias Políticas']},
        #                                     {'source': 'sub_subarea_ocde', 'subjects': ['Ciencias Políticas']},
        #                                    ],
        "classifications":[{"classification":str(GRU.get("TXT_CLASIF")),
                                                    "begin_date":str(GRU.get("DTA_CLASIF")),
                                                    "end_date":str(GRU.get("DTA_FIN_CLASIF")),
                                                    "source":"scienti"
                                                   }],
        "descriptions":{
            'PLAN_TRABAJO': GRU.get(f"TXT_PLAN_TRABAJO{borrar}"),
            'ESTADO_ARTE': GRU.get(f"TXT_ESTADO_ARTE{borrar}"),
            'OBJETIVOS': GRU.get(f"TXT_OBJETIVOS{borrar}"),
            'RETOS': GRU.get(f"TXT_RETOS{borrar}"),
            'VISION': GRU.get(f"TXT_VISION{borrar}"),
        },
        'programa_minciencias': {'programa':PRG.get("NME_PROGRAMA"),
                            'subdirección':PRG.get("NME_SUBDIRECCION")}    
        }

    if DEBUG: print('áreas ocde')

    if not AC.empty:
        subjects=[]
        stop=0 #Emergency stop
        STOP=False
        if AC.empty:
            STOP=True
        while not STOP:
            subjects.append({'title':AC.get("TXT_NME_AREA"),
                            'level':AC.get("NRO_NIVEL"),
                            'source': 'area_ocde'})    

            if AC.get('NRO_NIVEL')==0:
                STOP=True
            else:
                AC=ac[ac["COD_AREA_CONOCIMIENTO"]==AC.get("COD_AREA_PADRE")]
                AC=iloc_0(AC)

            #Emergency stop
            stop+=stop
            if stop>10:
                print("EMERGENCY EXIT in Area de Conocimiento")
                break
        if branch:
            branch['subjects']=subjects

    if aff:
        aff['branches']=branch
        pr['authors'][0]['affiliations']=[aff]
    else:
        pr['authors'][0]['affiliations']=[]

    prs.append(pr)
    if i%1000==0:
        f=open('products.json','w',encoding='utf8')
        json.dump(prs,f,cls=NpEncoder,ensure_ascii=False)
        f.close()
        
f=open('products.json','w',encoding='utf8')
json.dump(prs,f,cls=NpEncoder,ensure_ascii=False)
f.close() 

In [748]:
        f=open('products.json','w')
        json.dump(prs,f,cls=NpEncoder)
        f.close()    

In [765]:
f=open('products.json','r')
prs=json.load(f)
f.close()

In [767]:
df=pd.read_json('products.json',orient='records')

In [693]:
import json

In [699]:
[pr]

[{'year_published': None,
  'month_published': None,
  'source_checked': [{'source': 'Scienti',
    'COD_RH': '0000003069',
    'COD_PRODUCTO': 37}],
  'title': 'Blastocystis hominis: del ostracismo a la ilustración',
  'abstract': '',
  'notes': 'Blastocystis hominis: del ostracismo a la ilustración. Medellín, Colombia. Mayo de 2008 (Presentación oral). Primer Congreso Colombiano de Microbiología Clínica, Industrial, AmbientalAmbietal',
  'lang': 'ES',
  'product_subtype': [{'title': 'Completo', 'SGL_CATEGORIA': ''}],
  'product_type': [{'title': 'Producción bibliográfica', 'level': 0},
   {'title': 'Trabajos en eventos (Capítulos de memoria)', 'level': 1}],
  'keywords': [],
  'authors': [{'full_name': 'Gisela Maria Garcia Montoya',
    'first_names': 'Gisela Maria',
    'first_last_name': 'Garcia',
    'second_last_name': 'Montoya',
    'intials': 'GM',
    'aliases': ['GARCIA MONTOYA GISELA MARIA'],
    'sex': 'F',
    'nacionality': 'C',
    'marital_status': 'C',
    'date_of_bir

In [700]:
json.dump?

Signature:
json.dump(
    obj,
    fp,
    *,
    skipkeys=False,
    ensure_ascii=True,
    check_circular=True,
    allow_nan=True,
    cls=None,
    indent=None,
    separators=None,
    default=None,
    sort_keys=False,
    **kw,
)
Docstring:
Serialize ``obj`` as a JSON formatted stream to ``fp`` (a
``.write()``-supporting file-like object).

If ``skipkeys`` is true then ``dict`` keys that are not basic types
(``str``, ``int``, ``float``, ``bool``, ``None``) will be skipped
instead of raising a ``TypeError``.

If ``ensure_ascii`` is false, then the strings written to ``fp`` can
contain non-ASCII characters if they appear in strings contained in
``obj``. Otherwise, all such characters are escaped in JSON strings.

If ``check_circular`` is false, then the circular reference check
for container types will be skipped and a circular reference will
result in an ``OverflowError`` (or worse).

If ``allow_nan`` is false, then it will be a ``ValueError`` to
serialize out of range ``float`` 

In [666]:
subjects=[]
stop=0 #Emergency stop
STOP=False

while not STOP:
    subjects.append({'title':AC.get("TXT_NME_AREA"),
                    'level':AC.get("NRO_NIVEL"),
                    'source': 'area_ocde'})    
    
    if AC.get('NRO_NIVEL')==0:
        STOP=True
    else:
        AC=ac[ac["COD_AREA_CONOCIMIENTO"]==AC.get("COD_AREA_PADRE")]
        AC=iloc_0(AC)
        
    #Emergency stop
    stop+=stop
    if stop>10:
        print("EMERGENCY EXIT in Area de Conocimiento")
        break